https://github.com/kushalj001/pytorch-question-answering/blob/master/1.%20DrQA.ipynb

In [1]:
import pandas as pd
import numpy as np
import torchtext
import torch
from torch import nn
import json, re, unicodedata, string, typing, time
import torch.nn.functional as F
import spacy
from collections import Counter
import pickle
import Bert_Question_Answering_preprocessing as BQ
from nltk import word_tokenize
nlp = spacy.load('en')
%load_ext autoreload
%autoreload 2

In [6]:
train_data = BQ.load_json(r'C:\Users\abc\jupyter\pytorch\Bert\squad_train.json')
valid_data = BQ.load_json(r'C:\Users\abc\jupyter\pytorch\Bert\squad_dev.json')

train_list = BQ.parse_data(train_data)
valid_list = BQ.parse_data(valid_data)

print('Train list len : ', len(train_list))
print('Valid list lne : ', len(valid_list))

train_df = pd.DataFrame(train_list)
valid_df = pd.DataFrame(valid_list)

Length of data:  442
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  Beyoncé
Length of data:  35
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  Normans
Train list len :  86821
Valid list lne :  20302


In [7]:
def normalize_spaces(text):
    """
    정규화 과정
    """
    text = re.sub(r'\s', ' ',text)
    return text

train_df.context = train_df.context.apply(normalize_spaces)
valid_df.context = valid_df.context.apply(normalize_spaces)
                  

In [8]:
train_df.head()

,id,context,question,label,answer
0,56be85543aeaaa14008c9063,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"[269, 286]",in the late 1990s
1,56be85543aeaaa14008c9065,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,"[207, 226]",singing and dancing
2,56be85543aeaaa14008c9066,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,"[526, 530]",2003
3,56bf6b0f3aeaaa14008c9601,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"[166, 180]","Houston, Texas"
4,56bf6b0f3aeaaa14008c9602,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,"[276, 286]",late 1990s


In [11]:
%time vocab_text = BQ.gather_text_for_vocab([train_df, valid_df])
print("Number of sentences in dataset: ", len(vocab_text))

Wall time: 356 ms
Number of sentences in dataset:  112776


In [12]:
%time word2idx, idx2word, word_vocab = BQ.build_word_vocab(vocab_text)

raw-vocab: 108286
glove-vocab: 108286
vocab-length: 108288
word2idx-length: 108288
Wall time: 23.7 s


In [14]:
%time train_df['context_ids'] = train_df.context.apply(BQ.context_to_ids, word2idx=word2idx)
%time valid_df['context_ids'] = valid_df.context.apply(BQ.context_to_ids, word2idx=word2idx)

%time train_df['question_ids'] = train_df.question.apply(BQ.question_to_ids, word2idx = word2idx)
%time valid_df['question_ids'] = valid_df.question.apply(BQ.question_to_ids, word2idx = word2idx)

Wall time: 1min 8s
Wall time: 17.1 s
Wall time: 6.29 s
Wall time: 1.49 s


In [19]:
train_err = BQ.get_error_indices(train_df, idx2word)
valid_err = BQ.get_error_indices(valid_df, idx2word)

train_df.drop(train_err, inplace=True)
valid_df.drop(valid_err, inplace=True)

Error indices: 86821
Error indices: 20302
